# Data Analysis

## Linear Models
with `Statsmodels` package

In [2]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt

# for korean plotting
plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['axes.unicode_minus'] = False

# Pandas option
pd.set_option('display.max_columns', None)

In [3]:
# data load
df = pd.read_csv('df_preprocessed.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1671 entries, 0 to 1670
Data columns (total 40 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Bus             1671 non-null   float64
 1   Subway          1671 non-null   float64
 2   유흥업소            1671 non-null   float64
 3   살인              1671 non-null   int64  
 4   강도              1671 non-null   int64  
 5   강간              1671 non-null   int64  
 6   절도              1671 non-null   int64  
 7   폭력              1671 non-null   int64  
 8   방화              1671 non-null   int64  
 9   마약              1671 non-null   int64  
 10  약취              1671 non-null   int64  
 11  도박              1671 non-null   int64  
 12  전체범죄            1671 non-null   int64  
 13  112신고           1671 non-null   float64
 14  총_생활인구          1668 non-null   float64
 15  남성_생활인구         1668 non-null   float64
 16  여성_생활인구         1668 non-null   float64
 17  연령대_10_생활인구     1668 non-null   f

In [4]:
# Ordered Categorical Variable
cols_crime = ['살인', '강도', '강간', '절도', '폭력', '방화', '마약', '약취', '도박', '전체범죄']

t = pd.CategoricalDtype(categories=[1,2,3,4,5], ordered=True)
df[cols_crime] = df[cols_crime].astype(t)

In [5]:
df.columns[~df.columns.str.contains('생활인구')]

Index(['Bus', 'Subway', '유흥업소', '살인', '강도', '강간', '절도', '폭력', '방화', '마약', '약취',
       '도박', '전체범죄', '112신고', '아파트_단지수', '아파트_평균_시가', '개업율', '폐업율',
       '프랜차이즈_침투율', '매출액', '매출건수', '상권타입_골목상권', '상권타입_관광특구', '상권타입_발달상권',
       '상권타입_전통시장'],
      dtype='object')

In [6]:
# Endog, Exog, add Constatnt
X = df[df.columns[~df.columns.str.contains('생활인구')]]
X = sm.add_constant(X, prepend=True)
y = df['총_생활인구']

### OLS

In [7]:
# Model
model = sm.OLS(endog=y, exog=X, missing='drop')

In [8]:
# Result
res = model.fit()

In [9]:
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:                 총_생활인구   R-squared:                       0.673
Model:                            OLS   Adj. R-squared:                  0.667
Method:                 Least Squares   F-statistic:                     124.6
Date:                Sat, 15 Oct 2022   Prob (F-statistic):               0.00
Time:                        13:15:10   Log-Likelihood:                -1085.4
No. Observations:                1481   AIC:                             2221.
Df Residuals:                    1456   BIC:                             2353.
Df Model:                          24                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0235      0.062     -0.377      0.7

### VIF

In [10]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

vif = pd.DataFrame({'variable' : col_i, 'VIF' : variance_inflation_factor(model.exog, i)} for i, col_i in enumerate(model.exog_names) if col_i != 'const')
vif

/Users/dangchan/miniforge3/envs/torch/lib/python3.8/site-packages/statsmodels/stats/outliers_influence.py:195: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


,variable,VIF
0,Bus,1.696082
1,Subway,1.203912
2,유흥업소,1.302971
3,살인,1.353832
4,강도,1.392598
5,강간,5.082242
6,절도,8.304427
7,폭력,9.004269
8,방화,1.316199
9,마약,1.751470


In [11]:
# 전체범죄 제외 필요, 매출액/매출건수 조정 필요
X_new = X.drop(['전체범죄', '매출건수'], axis=1)

In [12]:
# Model_new
model_new = sm.OLS(endog=y, exog=X_new, missing='drop')

res_new = model_new.fit()
print(res_new.summary())

                            OLS Regression Results                            
Dep. Variable:                 총_생활인구   R-squared:                       0.666
Model:                            OLS   Adj. R-squared:                  0.661
Method:                 Least Squares   F-statistic:                     132.0
Date:                Sat, 15 Oct 2022   Prob (F-statistic):               0.00
Time:                        13:15:11   Log-Likelihood:                -1100.7
No. Observations:                1481   AIC:                             2247.
Df Residuals:                    1458   BIC:                             2369.
Df Model:                          22                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0161      0.063     -0.256      0.7

In [13]:
# retest VIF
from statsmodels.stats.outliers_influence import variance_inflation_factor

vif = pd.DataFrame({'variable' : col_i, 'VIF' : variance_inflation_factor(model_new.exog, i)} for i, col_i in enumerate(model_new.exog_names) if col_i != 'const')
vif ## VIF problem solved

/Users/dangchan/miniforge3/envs/torch/lib/python3.8/site-packages/statsmodels/stats/outliers_influence.py:195: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


,variable,VIF
0,Bus,1.685262
1,Subway,1.203592
2,유흥업소,1.301997
3,살인,1.347281
4,강도,1.392470
5,강간,4.687108
6,절도,4.883500
7,폭력,4.337966
8,방화,1.315709
9,마약,1.750576


#### Target Variable changing


In [14]:
# endog = 남성/여성생활인구?
# Model_men
model_men = sm.OLS(endog=df['남성_생활인구'], exog=X_new, missing='drop')

res_men = model_men.fit()
print(res_men.summary())

                            OLS Regression Results                            
Dep. Variable:                남성_생활인구   R-squared:                       0.663
Model:                            OLS   Adj. R-squared:                  0.658
Method:                 Least Squares   F-statistic:                     130.5
Date:                Sat, 15 Oct 2022   Prob (F-statistic):               0.00
Time:                        13:15:11   Log-Likelihood:                -1110.0
No. Observations:                1481   AIC:                             2266.
Df Residuals:                    1458   BIC:                             2388.
Df Model:                          22                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0234      0.063     -0.370      0.7

In [15]:
# Model_women
model_women = sm.OLS(endog=df['여성_생활인구'], exog=X_new, missing='drop')

res_women = model_women.fit()
print(res_women.summary())

                            OLS Regression Results                            
Dep. Variable:                여성_생활인구   R-squared:                       0.664
Model:                            OLS   Adj. R-squared:                  0.659
Method:                 Least Squares   F-statistic:                     131.1
Date:                Sat, 15 Oct 2022   Prob (F-statistic):               0.00
Time:                        13:15:12   Log-Likelihood:                -1100.0
No. Observations:                1481   AIC:                             2246.
Df Residuals:                    1458   BIC:                             2368.
Df Model:                          22                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0089      0.063     -0.141      0.8

## Creating New Features

In [16]:
# 매출건수당 매출금액(상권의 가격대 유추가능?)
from copy import deepcopy

X_add = deepcopy(X_new)
X_add['건수당매출'] = X_add['매출액'] / X_add['매출건수']
X_add = X_add.drop(['매출액', '매출건수'], axis=1)

KeyError: '매출건수'

In [ ]:
model_add = sm.OLS(endog=df['총_생활인구'], exog=X_add, missing='drop')

res_add = model_add.fit()
print(res_add.summary())

                            OLS Regression Results                            
Dep. Variable:                 총_생활인구   R-squared:                       0.572
Model:                            OLS   Adj. R-squared:                  0.566
Method:                 Least Squares   F-statistic:                     88.75
Date:                Fri, 14 Oct 2022   Prob (F-statistic):          4.44e-250
Time:                        13:33:46   Log-Likelihood:                -1488.0
No. Observations:                1481   AIC:                             3022.
Df Residuals:                    1458   BIC:                             3144.
Df Model:                          22                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -2.3818      0.423     -5.634      0.0

### 시간대별 생활인구 as Target Variable
시간대 1부터 6까지 각각 00~06시, 06~11시, 11~14시, 14~17시, 17~21시, 21~24시

In [ ]:
# 시간대별 생활인구?
# 심야시간대
model_time_6 = sm.OLS(endog=df['시간대_6_생활인구_수'], exog=X_new, missing='drop')

res_time_6 = model_time_6.fit()
print(res_time_6.summary())

                            OLS Regression Results                            
Dep. Variable:           시간대_6_생활인구_수   R-squared:                       0.659
Model:                            OLS   Adj. R-squared:                  0.654
Method:                 Least Squares   F-statistic:                     128.1
Date:                Sat, 15 Oct 2022   Prob (F-statistic):          3.94e-321
Time:                        09:18:10   Log-Likelihood:                -1082.7
No. Observations:                1481   AIC:                             2211.
Df Residuals:                    1458   BIC:                             2333.
Df Model:                          22                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0089      0.062     -0.142      0.8

## Feature Importance & Regression 종합
Gradient Boosting Machine의 

In [55]:
# load feature importance

df_imp = pd.read_csv('featureimportance.csv')
df_imp = df_imp.set_index('0')
df_imp.index.name = None
df_imp.columns = ['Feature_Importance']

In [58]:
df_res = df_imp.join([res_new.params[1:].rename('coef'),res_new.pvalues[1:].rename('p-values')])
df_res = df_res.sort_values(by='Feature_Importance', ascending=False)

In [59]:
df_res.round(4)

,Feature_Importance,coef,p-values
아파트_단지수,0.5720,0.4575,0.0000
매출액,0.1686,0.2729,0.0000
프랜차이즈_침투율,0.0356,-0.0169,0.2990
아파트_평균_시가,0.0346,-0.0404,0.0067
폐업율,0.0321,0.0406,0.0213
112신고,0.0290,0.0404,0.0067
개업율,0.0269,0.0108,0.5340
Bus,0.0248,0.1219,0.0000
유흥업소,0.0212,0.0608,0.0000
Subway,0.0200,0.1265,0.0000


In [60]:
# Feature Selection
df_res.loc[df_res['p-values']<0.1,:]

,Feature_Importance,coef,p-values
아파트_단지수,0.572046,0.457537,2.731397e-132
매출액,0.168585,0.272852,1.472742e-36
아파트_평균_시가,0.034599,-0.040427,6.695490e-03
폐업율,0.032058,0.040585,2.125167e-02
112신고,0.029027,0.040352,6.655427e-03
Bus,0.024823,0.121853,1.700083e-11
유흥업소,0.021155,0.060760,4.237058e-05
Subway,0.020020,0.126518,9.442545e-17
상권타입_전통시장,0.009269,-0.187991,3.283190e-03
폭력,0.006507,0.037353,4.583644e-02


Result : Bus, Subway, 유흥업소, 강간, 절도, 폭력, 방화, 약취, 112신고, 아파트_단지수, 아파트_평균_시가, 폐업율, 매출액, 상권타입